# importing

In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

## same with other team member dataset

### Dataset can be downloaded here: https://drive.google.com/file/d/1ipuh9aa36Da9fMC9O72pN-GfD5uo41A-/view?usp=sharing

In [3]:
df = pd.read_csv('D:/Tyler_Data/Other Skills/Datascience/HtetOps/Cancer_Predictive_Analytics/data/cancer_data_2004_2015.csv')

## dropping unrealted columns

In [4]:
df = df.drop(columns=['Histologic Type ICD-O-3','Grade Recode (thru 2017)','RX Summ--Scope Reg LN Sur (2003+)',
                      'RX Summ--Surg Oth Reg/Dis (2003+)',],axis=1)

## renaming columns

In [5]:
df = df.rename(columns={'Patient ID':'ID', 'Age recode with single ages and 85+':'age', 'Sex':'sex',
                        'Race recode (White, Black, Other)':'race', 'Marital status at diagnosis':'marital_status',
                        'Median household income inflation adj to 2023':'household_income','Rural-Urban Continuum Code':'location',
                        'Year of diagnosis':'year', 'Primary Site - labeled':'primary_site', 'CS tumor size (2004-2015)':'tumor_size',
                        'CS extension (2004-2015)':'tumor_spread', 'CS lymph nodes (2004-2015)':'tumor_lymph_nodes', 
                        'CS mets at dx (2004-2015)':'length_from_primary_diagnosis','Derived AJCC Stage Group, 6th ed (2004-2015)':'AJCC_stage',
                         'RX Summ--Surg Prim Site (1998+)':'surgery_primary', 'Radiation recode':'radiation','Chemotherapy recode (yes, no/unk)':'chemotherapy', 
                         'Survival months':'survival_month','COD to site recode':"cod", 'SEER cause-specific death classification':'cod_case',
                         'Vital status recode (study cutoff used)':'vital_status'})

## ID

In [6]:
print(df['ID'].nunique())
print(len(df['ID']))

1336877
1454497


In [7]:
# dropping duplicate of ID
df.drop_duplicates(subset=['ID'],inplace=True)

In [8]:
print(df['ID'].nunique())
print(len(df['ID']))

1336877
1336877


## Age

In [9]:
df['age']=df['age'].str.replace(' years','',regex=False)
df['age']=df['age'].str.replace('+','')
df['age']=df['age'].astype('int')

# collapsing primary site

In [10]:
# See all unique values in the column
pd.set_option('display.max_rows', None)   # show all rows
pd.set_option('display.max_columns', None)  # show all columns

# Example: view all unique categories
df['primary_site'].unique()

array(['C19.9-Rectosigmoid junction', 'C38.1-Anterior mediastinum',
       'C44.4-Skin of scalp and neck', 'C74.9-Adrenal gland, NOS',
       'C44.3-Skin other/unspec parts of face', 'C54.1-Endometrium',
       'C42.1-Bone marrow', 'C50.9-Breast, NOS',
       'C50.1-Central portion of breast', 'C65.9-Renal pelvis',
       'C16.0-Cardia, NOS', 'C61.9-Prostate gland', 'C22.0-Liver',
       'C44.8-Overlapping lesion of skin', 'C34.3-Lower lobe, lung',
       'C50.4-Upper-outer quadrant of breast',
       'C50.3-Lower-inner quadrant of breast',
       'C80.9-Unknown primary site',
       'C50.2-Upper-inner quadrant of breast', 'C18.2-Ascending colon',
       'C50.8-Overlapping lesion of breast',
       'C67.4-Posterior wall of bladder', 'C68.9-Urinary system, NOS',
       'C20.9-Rectum, NOS', 'C34.1-Upper lobe, lung', 'C64.9-Kidney, NOS',
       'C67.6-Ureteric orifice', 'C15.5-Lower third of esophagus',
       'C67.8-Overlapping lesion of bladder', 'C00.0-External upper lip',
       'C18.

In [11]:
def collapse_primary_site(value):
    code = value.split('-')[0]  # e.g., "C50.9"
    num = int(code[1:3])        # e.g., 50
    
    if 0 <= num <= 14:
        return "Lip/Oral/Pharynx"
    elif 15 <= num <= 26:
        return "Digestive system"
    elif 30 <= num <= 39:
        return "Respiratory system"
    elif 40 <= num <= 41:
        return "Bone"
    elif num == 42:
        return "Hematopoietic system"
    elif num == 44:
        return "Skin"
    elif 47 <= num <= 49:
        return "Soft tissue"
    elif num == 50:
        return "Breast"
    elif 51 <= num <= 58:
        return "Female genital organs"
    elif 60 <= num <= 63:
        return "Male genital organs"
    elif 64 <= num <= 68:
        return "Urinary system"
    elif 69 <= num <= 72:
        return "Eye/Brain/CNS"
    elif 73 <= num <= 75:
        return "Endocrine system"
    elif 76 <= num <= 80:
        return "Ill-defined/Unknown"
    else:
        return "Other"

# Apply mapping
df['PrimarySiteGroup'] = df['primary_site'].apply(collapse_primary_site)


In [12]:
df['PrimarySiteGroup'].nunique()


14

In [13]:
df['PrimarySiteGroup'].unique()

array(['Digestive system', 'Respiratory system', 'Skin',
       'Endocrine system', 'Female genital organs',
       'Hematopoietic system', 'Breast', 'Urinary system',
       'Male genital organs', 'Ill-defined/Unknown', 'Lip/Oral/Pharynx',
       'Eye/Brain/CNS', 'Soft tissue', 'Bone'], dtype=object)

# collapsing general cod

In [14]:
df['cod'].nunique()

94

In [15]:
df['cod'].unique()

array(['Alive', 'Trachea, Mediastinum and Other Respiratory Organs',
       'Other Cause of Death', 'Urinary Bladder', 'Corpus Uteri',
       'Myeloma', 'Diseases of Heart', 'Breast',
       'Kidney and Renal Pelvis', 'Alzheimers (ICD-9 and 10 only)',
       'Esophagus', 'Liver',
       'In situ, benign or unknown behavior neoplasm',
       'Non-Hodgkin Lymphoma', 'Lung and Bronchus',
       'Chronic Obstructive Pulmonary Disease and Allied Cond',
       'Other Urinary Organs',
       'Other Infectious and Parasitic Diseases including HIV',
       'Cerebrovascular Diseases', 'Prostate', 'Ureter',
       'Colon excluding Rectum', 'Other Digestive Organs',
       'Pneumonia and Influenza', 'Acute Myeloid Leukemia', 'Pancreas',
       'Uterus, NOS', 'Brain and Other Nervous System',
       'Nephritis, Nephrotic Syndrome and Nephrosis',
       'Rectum and Rectosigmoid Junction', 'Soft Tissue including Heart',
       'Miscellaneous Malignant Cancer', 'Ovary',
       'Intrahepatic Bile Duct'

In [16]:
def collapse_cod(value):
    if value == "Alive":
        return "Alive"
    
    # Cancer-specific
    cancer_cod = [
        "Lung and Bronchus", "Breast", "Prostate", "Colon excluding Rectum",
        "Rectum and Rectosigmoid Junction", "Kidney and Renal Pelvis",
        "Urinary Bladder", "Corpus Uteri", "Ovary", "Cervix Uteri",
        "Brain and Other Nervous System", "Thyroid", "Esophagus", "Liver",
        "Pancreas", "Stomach", "Gallbladder", "Small Intestine",
        "Non-Hodgkin Lymphoma", "Hodgkin Lymphoma", "Myeloma",
        "Acute Myeloid Leukemia", "Chronic Myeloid Leukemia",
        "Acute Lymphocytic Leukemia", "Chronic Lymphocytic Leukemia",
        "Other Myeloid/Monocytic Leukemia", "Other Lymphocytic Leukemia",
        "Acute Monocytic Leukemia", "Soft Tissue including Heart",
        "Bones and Joints", "Non-Melanoma Skin", "Melanoma of the Skin",
        "Nasopharynx", "Larynx", "Anus, Anal Canal and Anorectum",
        "Other Digestive Organs", "Other Oral Cavity and Pharynx",
        "Salivary Gland", "Pleura", "Testis", "Penis", "Vagina", "Vulva",
        "Eye and Orbit", "Retroperitoneum", "Peritoneum, Omentum and Mesentery",
        "Tonsil", "Tongue", "Lip", "Floor of Mouth", "Oropharynx",
        "Hypopharynx", "Nose, Nasal Cavity and Middle Ear", "Intrahepatic Bile Duct"
    ]
    
    if value in cancer_cod:
        return "Cancer-specific death"
    
    # Other disease
    other_disease = [
        "Diseases of Heart", "Cerebrovascular Diseases",
        "Aortic Aneurysm and Dissection", "Hypertension without Heart Disease",
        "Atherosclerosis", "Other Diseases of Arteries, Arterioles, Capillaries",
        "Chronic Obstructive Pulmonary Disease and Allied Cond",
        "Pneumonia and Influenza", "Tuberculosis",
        "Chronic Liver Disease and Cirrhosis", "Nephritis, Nephrotic Syndrome and Nephrosis",
        "Diabetes Mellitus", "Alzheimers (ICD-9 and 10 only)", "Septicemia",
        "Congenital Anomalies", "Stomach and Duodenal Ulcers",
        "Other Infectious and Parasitic Diseases including HIV",
        "Other Endocrine including Thymus"
    ]
    if value in other_disease:
        return "Other disease-related death"
    
    # External causes
    external = [
        "Accidents and Adverse Effects", "Suicide and Self-Inflicted Injury",
        "Homicide and Legal Intervention", "Complications of Pregnancy, Childbirth, Puerperium",
        "Certain Conditions Originating in Perinatal Period"
    ]
    if value in external:
        return "External causes"
    
    # Unknown/Ill-defined
    return "Unknown/Ill-defined"

# Apply collapse
df['COD_collapsed'] = df['cod'].apply(collapse_cod)


# dropping na

In [17]:
df = df.dropna()

In [18]:
df.isna().sum()

ID                               0
age                              0
sex                              0
race                             0
marital_status                   0
household_income                 0
location                         0
year                             0
primary_site                     0
tumor_size                       0
tumor_spread                     0
tumor_lymph_nodes                0
length_from_primary_diagnosis    0
AJCC_stage                       0
surgery_primary                  0
radiation                        0
chemotherapy                     0
survival_month                   0
cod                              0
cod_case                         0
vital_status                     0
PrimarySiteGroup                 0
COD_collapsed                    0
dtype: int64

# filtering dead data

In [19]:
df['vital_status'].unique()
df = df[df['vital_status']=='Dead']

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 646467 entries, 1 to 1454489
Data columns (total 23 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   ID                             646467 non-null  int64 
 1   age                            646467 non-null  int32 
 2   sex                            646467 non-null  object
 3   race                           646467 non-null  object
 4   marital_status                 646467 non-null  object
 5   household_income               646467 non-null  object
 6   location                       646467 non-null  object
 7   year                           646467 non-null  int64 
 8   primary_site                   646467 non-null  object
 9   tumor_size                     646467 non-null  int64 
 10  tumor_spread                   646467 non-null  int64 
 11  tumor_lymph_nodes              646467 non-null  int64 
 12  length_from_primary_diagnosis  646467 non-null  

# collapsing cause of death

In [21]:
cod_map = {'Cancer-specific death':'Cancer',
           'Other disease-related death': 'Other',
           'Unknown/Ill-defined':'Other',
           'External causes':'Other'}
df['COD_2'] = df['COD_collapsed'].map(cod_map)

In [22]:
df['COD_2'].value_counts()

COD_2
Cancer    396992
Other     249475
Name: count, dtype: int64

# collapsing location

In [23]:
df['location'].value_counts(dropna=False)

location
Counties in metropolitan areas ge 1 million pop                 311470
Counties in metropolitan areas of 250,000 to 1 million pop      173686
Nonmetropolitan counties adjacent to a metropolitan area         53938
Counties in metropolitan areas of lt 250 thousand pop            53816
Nonmetropolitan counties not adjacent to a metropolitan area     53249
Unknown/missing/no match/Not 1990-2023                             308
Name: count, dtype: int64

In [24]:
location_map = {'Counties in metropolitan areas ge 1 million pop':'metro',
                'Counties in metropolitan areas of 250,000 to 1 million pop':'metro',
                'Counties in metropolitan areas of lt 250 thousand pop':'metro',
                'Nonmetropolitan counties adjacent to a metropolitan area':'remote',
                'Nonmetropolitan counties not adjacent to a metropolitan area':'remote'}
df['location_collapsed']=df['location'].map(location_map)

In [25]:
df['location_collapsed'].value_counts(dropna=False)

location_collapsed
metro     538972
remote    107187
NaN          308
Name: count, dtype: int64

In [26]:
df['location_collapsed']=df['location_collapsed'].dropna()

In [27]:
df = df.dropna(subset=["location_collapsed"])

In [28]:
df['location_collapsed'].value_counts(dropna=False)

location_collapsed
metro     538972
remote    107187
Name: count, dtype: int64

# dropping year

In [29]:
df = df.drop('year',axis=1)

# dropping primary site

In [30]:
df = df.drop('primary_site',axis=1)

# dropping income and locatin

In [31]:
from pandas.api.types import CategoricalDtype

income_order = [
    "< $40,000",
    "$40,000 - $44,999",
    "$45,000 - $49,999",
    "$50,000 - $54,999",
    "$55,000 - $59,999",
    "$60,000 - $64,999",
    "$65,000 - $69,999",
    "$70,000 - $74,999",
    "$75,000 - $79,999",
    "$80,000 - $84,999",
    "$85,000 - $89,999",
    "$90,000 - $94,999",
    "$95,000 - $99,999",
    "$100,000 - $109,999",
    "$110,000 - $119,999",
    "$120,000+",
    ]

income_dtype = CategoricalDtype(categories=income_order, ordered=True)

df["income_cat"] = df["household_income"].astype(income_dtype)

In [32]:
df['income_cat'].value_counts(dropna=False,sort=False)

income_cat
< $40,000               2560
$40,000 - $44,999       2480
$45,000 - $49,999       5173
$50,000 - $54,999      15206
$55,000 - $59,999      20945
$60,000 - $64,999      42054
$65,000 - $69,999      49500
$70,000 - $74,999      40912
$75,000 - $79,999      75546
$80,000 - $84,999      59073
$85,000 - $89,999      67340
$90,000 - $94,999      59276
$95,000 - $99,999      83181
$100,000 - $109,999    67529
$110,000 - $119,999    41731
$120,000+              13653
Name: count, dtype: int64

In [33]:
df = df.drop(['household_income','location'],axis=1)

# filtering tumor size and spread

In [34]:
df['tumor_spread'].unique()
df['tumor_size'].nunique()
df.loc[df['tumor_size']>989,'tumor_size']=989

In [35]:
df.loc[df['tumor_spread']>989,'tumor_spread']=989

# dropping unnecessary columns

In [36]:
df = df.drop('length_from_primary_diagnosis',axis=1)

In [37]:
df=df.drop(['survival_month','cod','cod_case','vital_status','COD_collapsed'],axis=1)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 646159 entries, 1 to 1454489
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   ID                  646159 non-null  int64   
 1   age                 646159 non-null  int32   
 2   sex                 646159 non-null  object  
 3   race                646159 non-null  object  
 4   marital_status      646159 non-null  object  
 5   tumor_size          646159 non-null  int64   
 6   tumor_spread        646159 non-null  int64   
 7   tumor_lymph_nodes   646159 non-null  int64   
 8   AJCC_stage          646159 non-null  object  
 9   surgery_primary     646159 non-null  int64   
 10  radiation           646159 non-null  object  
 11  chemotherapy        646159 non-null  object  
 12  PrimarySiteGroup    646159 non-null  object  
 13  COD_2               646159 non-null  object  
 14  location_collapsed  646159 non-null  object  
 15  income_cat          6

# collapsing cancer stage

In [39]:
df['AJCC_stage'].unique()

array(['IB', 'IIC', 'IIIA', 'IIB', 'IIIB', 'III', 'II', 'UNK Stage',
       'IIA', '0is', 'I', 'IV', 'IA', 'IVA', 'IIIC', '0', '0a', 'IIIESB',
       'IVB', 'IB2', 'IEA', 'INOS', 'IEB', 'IC', 'OCCULT', 'IIEA', 'IVC',
       'IE', 'IIEB', 'IINOS', 'IIINOS', 'IIIEB', 'IIIEA', 'IIE', 'IVNOS',
       'IIIS', 'IA1', 'IA2', 'IIISA', 'IISB', 'IB1', 'IIS', 'ISA',
       'IIISB', 'IISA', 'IIIE', 'IIIESA', 'IS', 'IIESB', 'IIIES', 'IIESA',
       'IIES', 'ISB'], dtype=object)

In [40]:
import numpy as np

In [41]:
stage_map = {
    # Stage 0
    "0": "0", "0IS": "0", "0A": "0", "OCCULT": "0",
    # Stage I
    "I": "I", "IA": "I", "IA1": "I", "IA2": "I", "IB": "I", "IB1": "I", "IB2": "I", "IC": "I",
    "IS": "I", "ISB": "I", "IE": "I", "IEA": "I", "IEB": "I", "IEC": "I", "IE": "I", "ISA": "I",
    # Stage II
    "II": "II", "IIA": "II", "IIB": "II", "IIC": "II", "IIEA": "II", "IIEB": "II", "IIE": "II",
    "IIS": "II", "IISB": "II", "IISA": "II", "IIES": "II", "IIESA": "II", "IIESB": "II", "IINOS": np.nan,
    # Stage III
    "III": "III", "IIIA": "III", "IIIB": "III", "IIIC": "III", "IIIEA": "III", "IIIEB": "III", "IIIE": "III",
    "IIIES": "III", "IIIESA": "III", "IIIESB": "III", "IIIS": "III", "IIISB": "III",
    "IIISA": "III", "IIINOS": np.nan,
    # Stage IV
    "IV": "IV", "IVA": "IV", "IVB": "IV", "IVC": "IV", "IVNOS": np.nan,
    # Unknown / missing
    "UNK Stage": np.nan, "INOS": np.nan
}

# Apply mapping
df["AJCC_collapsed"] = df["AJCC_stage"].map(stage_map)

In [42]:
df['AJCC_collapsed'].value_counts(dropna=False)

AJCC_collapsed
IV     188679
II     133406
I      124486
III    104334
NaN     92092
0        3162
Name: count, dtype: int64

In [43]:
df= df.dropna()

In [44]:
df['AJCC_collapsed'].value_counts(dropna=False)

AJCC_collapsed
IV     188679
II     133406
I      124486
III    104334
0        3162
Name: count, dtype: int64

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 554067 entries, 1 to 1454486
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   ID                  554067 non-null  int64   
 1   age                 554067 non-null  int32   
 2   sex                 554067 non-null  object  
 3   race                554067 non-null  object  
 4   marital_status      554067 non-null  object  
 5   tumor_size          554067 non-null  int64   
 6   tumor_spread        554067 non-null  int64   
 7   tumor_lymph_nodes   554067 non-null  int64   
 8   AJCC_stage          554067 non-null  object  
 9   surgery_primary     554067 non-null  int64   
 10  radiation           554067 non-null  object  
 11  chemotherapy        554067 non-null  object  
 12  PrimarySiteGroup    554067 non-null  object  
 13  COD_2               554067 non-null  object  
 14  location_collapsed  554067 non-null  object  
 15  income_cat          5

In [46]:
df = df.drop('AJCC_stage',axis=1)

# collapsing surgery

In [47]:
df['surgery_primary'].unique()

array([60, 31, 33, 50, 80, 41, 21,  0, 30, 44, 27, 51, 40, 26, 25, 22, 45,
       57, 59, 23, 20, 52, 98, 32, 55, 35, 42, 37, 70, 90, 49, 16, 14, 11,
       24, 61, 71, 19, 54, 28, 36, 47, 53, 46, 67, 56, 15, 34, 72, 65, 12,
       66, 64, 43, 62, 77, 13, 48, 10, 38, 63, 99, 29, 73, 17, 69, 58, 75,
       74, 78, 76, 18, 68], dtype=int64)

In [48]:
df.loc[df['surgery_primary']>0,'surgery_primary']=1

In [49]:
# 0 for no, 1 for yes
df['surgery_primary'].value_counts(dropna=False)

surgery_primary
0    285482
1    268585
Name: count, dtype: int64

# radiation

In [50]:
yes_set = {
    "Beam radiation",
    "Radioactive implants (includes brachytherapy) (1988+)",
    "Combination of beam with implants or isotopes",
    "Radioisotopes (1988+)"
}

no_set = {
    "None/Unknown",
    "Refused (1988+)",
    "Recommended, unknown if administered",
    "Radiation, NOS  method or source not specified"
}

df["radiation_collapsed"] = df["radiation"].apply(lambda x: 1 if x in yes_set else 0)

In [51]:
df['radiation_collapsed'].value_counts()

radiation_collapsed
0    384506
1    169561
Name: count, dtype: int64

In [52]:
df = df.drop('radiation',axis=1)

In [53]:
df = df.drop('ID',axis=1)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 554067 entries, 1 to 1454486
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   age                  554067 non-null  int32   
 1   sex                  554067 non-null  object  
 2   race                 554067 non-null  object  
 3   marital_status       554067 non-null  object  
 4   tumor_size           554067 non-null  int64   
 5   tumor_spread         554067 non-null  int64   
 6   tumor_lymph_nodes    554067 non-null  int64   
 7   surgery_primary      554067 non-null  int64   
 8   chemotherapy         554067 non-null  object  
 9   PrimarySiteGroup     554067 non-null  object  
 10  COD_2                554067 non-null  object  
 11  location_collapsed   554067 non-null  object  
 12  income_cat           554067 non-null  category
 13  AJCC_collapsed       554067 non-null  object  
 14  radiation_collapsed  554067 non-null  int64   
dtypes: c

In [55]:
df.to_csv('final_data.csv',index=False)